In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl


# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
from matplotlib.ticker import ScalarFormatter


In [2]:
# 데이터 불러오기
gj = pd.read_excel('data/거제벨버디어.xlsx')
npl = pd.read_excel('data/나폴리농원.xlsx')
dhs = pd.read_excel('data/대흥사.xlsx')
st = pd.read_excel('data/센텀스파랜드.xlsx')
scm = pd.read_excel('data/순천만국가정원.xlsx')
cdb = pd.read_excel('data/차덕분.xlsx')
forest = pd.read_excel('data/포레스트리솜.xlsx')
hb = pd.read_excel('data/한방진흥원.xlsx')
hbc = pd.read_excel('data/해비치.xlsx')

In [3]:
review = pd.concat([gj, npl, dhs, st, scm, cdb, forest, hb, hbc], ignore_index=True)
review.tail()
review.columns

Index(['리뷰', '작성자', '작성자정보', '날짜', '방문회수', '인증수단', '업소명', '지역', '작성일', '방문횟수'], dtype='object')

In [4]:
# 작성자, 작정자정보, 방문횟수, 인증수단, 작성일, 방문횟수 -> 제거
review = review.drop(columns=['작성자', '작성자정보', '방문회수', '인증수단', '작성일', '방문횟수'])
review.tail() #14390개 데이터

,리뷰,날짜,업소명,지역
14386,ㅇ에,2013년 10월 24일 목요일,해비치 호텔&리조트 제주,제주
14387,숙박,2013년 10월 24일 목요일,해비치 호텔&리조트 제주,제주
14388,숙박,2013년 10월 24일 목요일,해비치 호텔&리조트 제주,제주
14389,짱,2013년 10월 24일 목요일,해비치 호텔&리조트 제주,제주
14390,좋아요,201년 4월 27일 월요일,해비치 호텔&리조트 제주,제주


In [5]:
# 날짜 형식 변환
# 1. 연, 월, 일 숫자만 추출
review[['연', '월', '일']] = review['날짜'].str.extract(r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일')

# 2. 추출한 숫자로 날짜 문자열 생성 (YYYY-MM-DD)
review['날짜_문자'] = review.apply(
    lambda row: f"{int(row['연']):04d}-{int(row['월']):02d}-{int(row['일']):02d}" 
    if pd.notnull(row['연']) and pd.notnull(row['월']) and pd.notnull(row['일']) 
    else None, axis=1)

# 3. 날짜 문자열을 datetime 타입으로 변환
review['날짜'] = pd.to_datetime(review['날짜_문자'], format='%Y-%m-%d', errors='coerce')

# 4. 임시 컬럼 삭제
review.drop(columns=['연', '월', '일', '날짜_문자'], inplace=True)

# 결과 확인
print(review['날짜'].head())

# 데이터 저장: 계속 이 코드에서 오류가 나서 억지로 데이터를 저장하고 다시 시작
review.to_csv('data/review_processed.csv', index=False, encoding='utf-8-sig')   

0   2025-07-19
1   2025-07-05
2   2025-05-30
3   2025-07-11
4   2025-06-02
Name: 날짜, dtype: datetime64[ns]


In [6]:
# 데이터 불러오기
r2 = pd.read_csv('data/review_processed.csv', encoding='utf-8-sig ')
len(r2) #14391개

14391

In [7]:
r2['날짜'].dtype #object
# r2['날짜'] = pd.to_datetime(r2['날짜'], errors='coerce')
# r2 = r2.dropna(subset=['날짜']) 
# r2['등록일'] = r2['날짜'].dt.strftime('%Y%m%d').astype(int)
# len(r2) #8670개

dtype('O')

In [16]:
# 날짜 제한 (2023.01.01 ~ 2025.07.31)까지로 제한
# 다른 데이터들과 어느정도 맞추기 위해서
r2 = r2[(r2['날짜'] >= '2023-01-01') & (r2['날짜'] <= '2025-07-31')]
len(r2) #2779개

2779

In [14]:
# 최종 데이터 저장
r2.to_csv('data/review_final.csv', index=False, encoding='utf-8-sig')